In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import re
import json
import requests
from tabulate import tabulate
from datetime import datetime, timedelta
import time
import ast

In [2]:
def get_access_token(auth_url, client_id, client_secret):
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    response = requests.post(auth_url, data=data)
    json_response = response.json()
    return json_response["access_token"]

def get_types(base_url, headers):
    endpoint = "/cases/types"
    response = requests.get(f"{base_url}{endpoint}", headers=headers)
    response.raise_for_status()
    print(response)
    return response.json()

#Returns a list of unique jurisdiction names

def extract_karbon_excel(cutoff=None):
    auth_url = "https://auth.dev.karbon.deltacapita.net/oauth2/token"
    client_id = "6kjogklf4s4370ebpre78mfg79"
    client_secret = "12biur7mk3f6q1jc70en2li20hkstc6emnkelvsi0gm4hhrrpf09"
    base_url = "https://api.dev.karbon.deltacapita.net"
    
    access_token = get_access_token(auth_url, client_id, client_secret)
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    # Get all types
    types = get_types(base_url, headers)
    print(f"Retrieved {len(types)} types")
    all_results = []
    max_retries = 10
    retry_delay = 20  # seconds

    for type in types:

        print(f"Fetching cases for type: {type}")
        
        result_count = 400
        type_results = []
        more_results = True

        while more_results:
            for attempt in range(max_retries):
                try:
                    response = requests.get(
                        f"{base_url}/cases",
                        headers=headers,
                        params={'type': type, 'resultCount': result_count},
                        timeout=30
                    )
                    response.raise_for_status()
                    data = response.json()
                    results = data.get('results', [])
                    
                    print(f"Retrieved {len(results)} cases for {type} (requested {result_count})")
                    
                    type_results = results  # Replace instead of extend
                    
                    if len(results) < result_count:
                        more_results = False
                    else:
                        result_count += 400
                        print(f"Increasing result count to {result_count} for {type}")

                    break  # Successful request, exit retry loop

                except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
                    print(f"Error occurred for {type}: {e}. Attempt {attempt + 1} of {max_retries}")
                    if attempt == max_retries - 1:
                        print(f"Failed to retrieve data for {type} after {max_retries} attempts. Skipping.")
                    time.sleep(retry_delay)
            else:
                more_results = False  # Exit if all retries are exhausted

        print(f"Total cases for {type}: {len(type_results)}")
        all_results.extend(type_results)

    df = pd.DataFrame(all_results)
    
    def safe_get(dictionary, key):
        if isinstance(dictionary, dict):
            return dictionary.get(key)
        return None

    keys_to_extract = ['jurisdiction', 'customerInternalID']

    if 'entities' in df.columns:
        for key in keys_to_extract:
            df[key] = df['entities'].apply(lambda x: safe_get(x[0] if isinstance(x, list) and len(x) > 0 else {}, key))
        df = df.drop(columns=['entities'])
    else:
        for key in keys_to_extract:
            if key not in df.columns:
                print(f"Warning: '{key}' not found in the data.")
    df.to_excel('Test.xlsx', index=False)
    df['updatedDateTime'] = pd.to_datetime(df['updatedDateTime'], format='%d-%m-%Y %H:%M:%S')
    df['createDateTime'] = pd.to_datetime(df['createDateTime'], format='%d-%m-%Y %H:%M:%S')
    if cutoff is not None:
        if not isinstance(cutoff, pd.Timestamp):
            cutoff = pd.Timestamp(cutoff)

        cutoff_df = df[(df['createDateTime'] >= cutoff)| (df['updatedDateTime'] >= cutoff)]
    else:
        cutoff_df = None
    df.to_excel('example_extract.xlsx', index=False)
    return df, cutoff_df

In [3]:
def get_history(base_url, headers, case_id, max_retries=3):
    for attempt in range(max_retries):
        try:
            endpoint = f"/cases/{case_id}/history"
            response = requests.get(f"{base_url}{endpoint}", headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed for case {case_id}: {e}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2 ** attempt)  # Exponential backoff

#Returns the audit logs for a given case

def extract_updated_values(action_type):
    try:
        start = action_type.index("updated values:") + len("updated values:")
        end = action_type.index("previous values:")
        updated_values_str = action_type[start:end].strip()
        
        # Remove the leading and trailing parentheses if present
        updated_values_str = updated_values_str.strip('()')
        
        # Split the string by comma, but not within parentheses
        values = []
        current = ''
        paren_count = 0
        for char in updated_values_str:
            if char == '(' or char == '[':
                paren_count += 1
            elif char == ')' or char == ']':
                paren_count -= 1
            
            if char == ',' and paren_count == 0:
                values.append(current.strip())
                current = ''
            else:
                current += char
        
        if current:
            values.append(current.strip())
        
        # Convert string representations to actual Python objects
        converted_values = []
        for value in values:
            try:
                converted_values.append(ast.literal_eval(value))
            except:
                converted_values.append(value)
        
        return tuple(converted_values)
    except Exception as e:
        print(f"Error in extract_updated_values: {e}")
        return None

#Takes the audit log event action type, returns the updated values

def get_update_create_date_by_risk(base_url, headers, case_id):
    for attempt in range(2):
        try:
            endpoint = f"/cases/{case_id}"
            response = requests.get(f"{base_url}{endpoint}", headers=headers)
            response.raise_for_status()
            
            # Parse the JSON response
            case_data = response.json()
            
            # Extract createDateTime and createdBy
            update_date_time = case_data.get('updatedDateTime')
            updated_by = case_data.get('updatedBy')
            create_date_time = case_data.get('createDateTime')
            created_by = case_data.get('createdBy')
            riskLevel = case_data.get('riskLevel')
            
            # Convert createDateTime to a datetime object if it's not None
            if update_date_time:
                update_date_time = datetime.strptime(update_date_time, '%d-%m-%Y %H:%M:%S')
            if create_date_time:
                create_date_time = datetime.strptime(create_date_time, '%d-%m-%Y %H:%M:%S')
            
            # If we reach here, it means the request was successful
            # So we return the data and break out of the loop
            return update_date_time, updated_by, create_date_time, created_by, riskLevel

        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed for case {case_id}: {e}")
            if attempt == 2:  # This is the last attempt
                raise
            time.sleep(2 ** attempt)  # Exponential backoff
    
    # If we've exhausted all attempts without success, return None
    return None, None, None, None, None

#Takes a case id, then returns the updatedDateTime, updatedBy, createDateTime, createdBy, riskLevel

def find_status_change_times(history, status_from, status_to):
    acceptance_required_time = None
    accepted_time = None
    oldest_timestamp = None

    for event in reversed(history):
        eventdetail = event.get('eventdetail', {})
        action_type = eventdetail.get('action type', '')
        timestamp = event.get('timestamp')
        
        # Keep track of the oldest timestamp
        if oldest_timestamp is None or timestamp < oldest_timestamp:
            oldest_timestamp = timestamp

        if "updated values:" in action_type:
            updated_values = extract_updated_values(action_type)
            if updated_values and len(updated_values) >= 1:
                new_status = updated_values[0]
                
                if new_status == status_to and accepted_time is None:
                    accepted_time = datetime.strptime(timestamp, '%d-%m-%Y %H:%M:%S')
                elif new_status == status_from and acceptance_required_time is None:
                    acceptance_required_time = datetime.strptime(timestamp, '%d-%m-%Y %H:%M:%S')
                    
                # If we've found both times, we can stop searching
                if acceptance_required_time and accepted_time:
                    break

    # If we didn't find an explicit 'Case Accepted' status change,
    # assume the case was created as 'Case Accepted'
    if not accepted_time and oldest_timestamp:
        accepted_time = datetime.strptime(oldest_timestamp, '%d-%m-%Y %H:%M:%S')

    return acceptance_required_time, accepted_time

#Takes in a target original status, returns the most recent occurance time the case status moved to 'status_from' and time the case moved to 'status_to'

def find_status_updateBy_risk(history, status_to):
    updated_by = None
    risk_level = None
    for event in reversed(history):  # We reverse to check the most recent events first
        eventdetail = event.get('eventdetail', {})
        action_type = eventdetail.get('action type', '')
        
        if "updated values:" in action_type and f"{status_to}" in action_type:
            updated_values = extract_updated_values(action_type)
            previous_values = eventdetail.get('previous values', {})
            
            # Get updatedBy from previous values
            updated_by = previous_values.get('updatedBy')
            
            # Get riskLevel from previous values
            risk_level = previous_values.get('riskLevel')
            
            
            break  # We found the 'Case Accepted' update, so we can stop searching
    return updated_by, risk_level

#Takes in a history, target status to, and returns the riskLevel of the case, at the time of the most recent occurance of the case moving to that status, along with updatedBy when the status was changed.

def extract_status_change_info(case_id, status_from, status_to):
    auth_url = "https://auth.dev.karbon.deltacapita.net/oauth2/token"
    client_id = "6kjogklf4s4370ebpre78mfg79"
    client_secret = "12biur7mk3f6q1jc70en2li20hkstc6emnkelvsi0gm4hhrrpf09"
    base_url = "https://api.dev.karbon.deltacapita.net"
    
    access_token = get_access_token(auth_url, client_id, client_secret)
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    accepted_time = None
    time_difference = None
    updated_by = None
    risk_level = None
    acceptance_required_time = None
    try:
        history = get_history(base_url, headers, case_id)
        acceptance_required_time, accepted_time = find_status_change_times(history, status_from, status_to)
        updated_by, risk_level = find_status_updateBy_risk(history, status_to)
            
        if accepted_time:
            if acceptance_required_time:
                time_difference = accepted_time - acceptance_required_time
            else:
                time_difference = f"N/A (Created as {status_to})"
        if not updated_by:
            updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[1]
        if not updated_by:
            updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[3]    
        if not risk_level:
            risk_level = get_update_create_date_by_risk(base_url, headers, case_id)[4]
        if not accepted_time:
            accepted_time = get_update_create_date_by_risk(base_url, headers, case_id)[0]
        if not accepted_time:
            accepted_time = get_update_create_date_by_risk(base_url, headers, case_id)[2]
        if not acceptance_required_time:
            acceptance_required_time = get_update_create_date_by_risk(base_url, headers, case_id)[2]
            

        print(f"Case ID: {case_id}")
        print(f"Time from {status_from} to {status_to}: {time_difference}")
        print(f"Date status moved to {status_to}: {accepted_time}")
        print(f"Updated By: {updated_by}")
        print(f"Risk Level: {risk_level}")

    except requests.exceptions.HTTPError as e:
        print(f"Failed to retrieve history for case {case_id}: {e}")
        updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[1]
        if not updated_by:
            updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[3] 
        accepted_time = get_update_create_date_by_risk(base_url, headers, case_id)[0]
        if not accepted_time:
            accepted_time = get_update_create_date_by_risk(base_url, headers, case_id)[2]
        risk_level = get_update_create_date_by_risk(base_url, headers, case_id)[4]
        time_difference = f"N/A (Created as {status_to})"
        acceptance_required_time = get_update_create_date_by_risk(base_url, headers, case_id)[2]
        
        
    print()
    return accepted_time, time_difference, updated_by, risk_level, acceptance_required_time

'''Takes in the case id, status_from, status_to:
1.) Checks the audit log to see if it includes the update when case status was moved:
        If it does, then we return the time taken to go from status_from to status_to.
        If it doesn't, we then check case 'updatedDateTime' and return that.
        If it doesn't have that, we assume the case was create with status_from, so return created time.
2.) Checks the audit log for details on who made the update:    
        If it includes details on the specific change we use that.
        If it doesn't then we check the case 'updatedBy' and return that.
        If there is no 'updatedBy' then we just return 'createdBy'.
3.) Checks the audit log for details on the risk at the time of the update:    
        If it includes details on the specific change we use that.
        If it doesn't then we check the case 'riskLevel' and return that.
''' 

"Takes in the case id, status_from, status_to:\n1.) Checks the audit log to see if it includes the update when case status was moved:\n        If it does, then we return the time taken to go from status_from to status_to.\n        If it doesn't, we then check case 'updatedDateTime' and return that.\n        If it doesn't have that, we assume the case was create with status_from, so return created time.\n2.) Checks the audit log for details on who made the update:    \n        If it includes details on the specific change we use that.\n        If it doesn't then we check the case 'updatedBy' and return that.\n        If there is no 'updatedBy' then we just return 'createdBy'.\n3.) Checks the audit log for details on the risk at the time of the update:    \n        If it includes details on the specific change we use that.\n        If it doesn't then we check the case 'riskLevel' and return that.\n"

In [4]:
def find_status_change_and_details(history, current_status):
    for event in reversed(history):
        eventdetail = event.get('eventdetail', {})
        action_type = eventdetail.get('action type', '')
        timestamp = event.get('timestamp')
        
        if "updated values:" in action_type and "previous values:" in action_type:
            updated_values = extract_updated_values(action_type)
            if updated_values and len(updated_values) >= 1:
                new_status = updated_values[0]
                
                # Extract previous values
                prev_start = action_type.index("previous values:") + len("previous values:")
                prev_values_str = action_type[prev_start:].strip()
                prev_values = ast.literal_eval(prev_values_str)
                
                old_status = prev_values.get('status')
                
                if new_status == current_status and old_status != current_status:
                    change_time = datetime.strptime(timestamp, '%d-%m-%Y %H:%M:%S')
                    updated_by = prev_values.get('updatedBy')
                    risk_level = prev_values.get('riskLevel')
                    return change_time, updated_by, risk_level
    
    return None, None, None

#Takes in history, along with to_status, returns the time of the most recent occurance that the case was moved to this status, along with who did (if detailed in logs), along with the risk level at that time.



    

def extract_status_to_info(case_id, status_to):
    auth_url = "https://auth.dev.karbon.deltacapita.net/oauth2/token"
    client_id = "6kjogklf4s4370ebpre78mfg79"
    client_secret = "12biur7mk3f6q1jc70en2li20hkstc6emnkelvsi0gm4hhrrpf09"
    base_url = "https://api.dev.karbon.deltacapita.net"
    
    access_token = get_access_token(auth_url, client_id, client_secret)
    
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    
    change_time = None
    updated_by = None
    risk_level = None
    try:
        history = get_history(base_url, headers, case_id)
        change_time, updated_by, risk_level = find_status_change_and_details(history, status_to)
            
        if not updated_by:
            updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[1]
        if not updated_by:
            updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[3]    
        if not risk_level:
            risk_level = get_update_create_date_by_risk(base_url, headers, case_id)[4]
        if not change_time:
            change_time = get_update_create_date_by_risk(base_url, headers, case_id)[0]
        if not change_time:
            change_time = get_update_create_date_by_risk(base_url, headers, case_id)[2]
            

        print(f"Case ID: {case_id}")
        print(f"Date status moved to {status_to}: {change_time}")
        print(f"Updated By: {updated_by}")
        print(f"Risk Level: {risk_level}")

    except requests.exceptions.HTTPError as e:
        print(f"Failed to retrieve history for case {case_id}: {e}")
        updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[1]
        if not updated_by:
            updated_by = get_update_create_date_by_risk(base_url, headers, case_id)[3] 
        change_time = get_update_create_date_by_risk(base_url, headers, case_id)[0]
        if not change_time:
            change_time = get_update_create_date_by_risk(base_url, headers, case_id)[2]
        risk_level = get_update_create_date_by_risk(base_url, headers, case_id)[4]
        
    print()
    return change_time, updated_by, risk_level

'''Takes in the case id, status_to:
1.) Checks the audit log to see if it includes the update when case status was moved to status_to:
        If it does, then we return the time taken to go to status_to.
        If it doesn't, we then check case 'updatedDateTime' and return that.
        If it doesn't have that, we assume the case was create with status_from, so return created time.
2.) Checks the audit log for details on who made the update:    
        If it includes details on the specific change we use that.
        If it doesn't then we check the case 'updatedBy' and return that.
        If there is no 'updatedBy' then we just return 'createdBy'.
3.) Checks the audit log for details on the risk at the time of the update:    
        If it includes details on the specific change we use that.
        If it doesn't then we check the case 'riskLevel' and return that.
''' 
extract_status_to_info('7b630e20-f002-4363-b273-3afd27f3c0eb', 'Confirmed - False Positive')


Case ID: 7b630e20-f002-4363-b273-3afd27f3c0eb
Date status moved to Confirmed - False Positive: 2024-04-29 16:17:27
Updated By: sylvia.ingle
Risk Level: Low



(datetime.datetime(2024, 4, 29, 16, 17, 27), 'sylvia.ingle', 'Low')

In [5]:
def create_employee_metrics_df(df):
    '''Creates a df of employee performance metrics based on initial df input'''

    # Extract date
    last_updated = pd.to_datetime('today')  

    # Read the excel file of initial df

    # Create list of unique names
    persons = df['updatedBy'].dropna().unique()

    
    # Create range of months
    months = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
    # Create a list of all combinations of months and persons
    data = [(month, person) for month in months for person in persons]
    # Create the DataFrame
    person_month_df = pd.DataFrame(data, columns=['Date', 'Employee'])
    # Add the additional columns with zeros
    person_month_df['case_acc_count'] = 0
    person_month_df['case_screen_count'] = 0
    person_month_df['case_create_count'] = 0
    # Sort the DataFrame by Month and then by Employee
    person_month_df = person_month_df.sort_values(['Date', 'Employee']).reset_index(drop=True)

    relevant_status = ['Case Accepted', 'Case with Waiver Accepted']
    df_acc_users = df[(df['status'].isin(relevant_status)) & (df['type']=='New') & (df['customerInternalID'].notna())].copy()
    acc_id = df_acc_users['id'].tolist()
    
    for id in acc_id:
        acc_type = df[df['id']==id]['status'].iloc[0]
        if acc_type == 'Case with Waiver Accepted':
            status_from, status_to = 'Case with Waiver Acceptance Required', 'Case with Waiver Accepted'
        else:
            status_from, status_to = 'Case Acceptance Required', 'Case Accepted'
        info = extract_status_change_info(id, status_from=status_from, status_to=status_to)
        df_acc_users.loc[df_acc_users['id']==id, 'acceptDateTime'] = info[0]
        df_acc_users.loc[df_acc_users['id']==id, 'acceptTimeDelta'] = info[1]
        df_acc_users.loc[df_acc_users['id']==id, 'acceptedBy'] = info[2]
        df_acc_users.loc[df_acc_users['id']==id, 'accreqDateTime'] = info[4]
        df_acc_users.loc[df_acc_users['id']==id, 'accreqriskLevel'] = info[3]
        print(acc_type)
        
    # Create year-month column
    df_acc_users['year_month'] = df_acc_users['acceptDateTime'].dt.to_period('M')

    # Count cases per month for each user
    monthly_counts = df_acc_users.groupby(['year_month', 'acceptedBy']).size().reset_index(name='count')

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')

    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        update_by = row['acceptedBy']
        count = row['count']
        
        # Convert year_month to datetime for column indexing
        month_date = year_month.to_timestamp()
        
        # Update the count for the specific user and month
        mask = (person_month_df['Date'] == month_date) & (person_month_df['Employee'] == update_by)
        person_month_df.loc[mask, 'case_acc_count'] = count
    
    
    relevant_status = ['Confirmed - False Positive', 'Confirmed - Positive']
    df_screen_users = df[(df['status'].isin(relevant_status)) & (df['customerInternalID'].notna())].copy()
    screen_id = df_screen_users['id'].tolist()

    for id in screen_id:
        screen_type = df[df['id']==id]['status'].iloc[0]
        info = extract_status_to_info(id, status_to=screen_type)
        df_screen_users.loc[df_screen_users['id']==id, 'screenDateTime'] = info[0]
        df_screen_users.loc[df_screen_users['id']==id, 'screenedBy'] = info[1]
        print(screen_type)
    df_screen_users.to_excel('Test_screen.xlsx', index=False)
    '''For each person, count the number of cases screened per month.'''
    # Create year-month column

    df_screen_users['screenDateTime'] = pd.to_datetime(df_screen_users['screenDateTime'])

    df_screen_users['year_month'] = df_screen_users['screenDateTime'].dt.to_period('M')

    # Count cases per month for each user
    monthly_counts = df_screen_users.groupby(['year_month', 'screenedBy']).size().reset_index(name='count')

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')
    
    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        update_by = row['screenedBy']
        count = row['count']
        
        # Convert year_month to datetime for column indexing
        month_date = year_month.to_timestamp()
        
        # Update the count for the specific user and month
        mask = (person_month_df['Date'] == month_date) & (person_month_df['Employee'] == update_by)
        person_month_df.loc[mask, 'case_screen_count'] = count
    
    df_new_users = df[(df['type']=='New') & (df['customerInternalID'].notna())].copy()

    '''For each person, count the number of cases screened per month.'''
    # Create year-month column

    df_new_users['createDateTime'] = pd.to_datetime(df_new_users['createDateTime'])

    df_new_users['year_month'] = df_new_users['createDateTime'].dt.to_period('M')

    # Count cases per month for each user
    monthly_counts = df_new_users.groupby(['year_month', 'createdBy']).size().reset_index(name='count')

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')
    
    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        update_by = row['createdBy']
        count = row['count']
        
        # Convert year_month to datetime for column indexing
        month_date = year_month.to_timestamp()
        
        # Update the count for the specific user and month
        mask = (person_month_df['Date'] == month_date) & (person_month_df['Employee'] == update_by)
        person_month_df.loc[mask, 'case_create_count'] = count
    #Replace existing file
    person_month_df.to_excel('excel_files/employee_performance_metrics.xlsx', index=False)
    return df_acc_users, df_screen_users
    # Export as an excel file


In [6]:
def create_SLA_df(df_acc_users):
    
    SLA_df = df_acc_users.copy()

    #Iterate over the rows in updated cases
    for index, row in SLA_df.iterrows():
        case_id = row['id']
        
        # Convert float to Timedelta if necessary
        if isinstance(row['acceptTimeDelta'], float):
            time_to_accept = pd.to_timedelta(row['acceptTimeDelta'], unit='D')
        elif isinstance(row['acceptTimeDelta'], pd.Timedelta):
            time_to_accept = row['acceptTimeDelta']
        else:
            SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = "Created as Case Accepted"
            continue

        #Create suitable timedelta objects for comparison
        one_month = pd.Timedelta(days=30)
        two_months = pd.Timedelta(days=60)
        three_months = pd.Timedelta(days=90)

        if pd.notna(row['riskLevel']):
            if row['riskLevel'] == 'Low':
                if time_to_accept < one_month:
                    SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = "Flagged: Low risk but took less than 1 month."
                else:
                    SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = "Not Flagged"
            elif row['riskLevel'] == 'Standard':
                if time_to_accept < two_months:
                    SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = "Flagged: Standard risk but took less than 2 months."
                else:
                    SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = "Not Flagged"
            elif row['riskLevel'] == 'Heightened':
                if time_to_accept < three_months:
                    SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = "Flagged: Heightened risk but took less than 3 months."
                else:
                    SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = "Not Flagged"
            else:
                SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = f"Unknown risk level: {row['riskLevel']}"
        else:
            SLA_df.loc[SLA_df['id'] == case_id, 'SLAscore'] = 'No riskLevel for case.'

    '''Creates a df of SLA 'score' based on initail DB input'''

    #Read the excel file of initial DB
    
    #Create list of unique names
    scores = SLA_df['SLAscore'].dropna().unique()

    #Create range of months
    months = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')

    #Create a new df of people against months, with zeroes in entries
    data = [(month, score) for month in months for score in scores]

    score_month_df = pd.DataFrame(data, columns=['Date', 'SLAscore'])

    score_month_df['case_count'] = 0
    
    score_month_df = score_month_df.sort_values(['Date', 'SLAscore']).reset_index(drop=True)

    '''For each person, count the number of SLA scores per month.'''

    # Read the excel files
    
    score_month_df['Date'] = pd.to_datetime(score_month_df['Date'])

    # Turn date-time column into a pd.datetime element
    SLA_df['acceptDateTime'] = pd.to_datetime(SLA_df['acceptDateTime'])

    # Create year-month column
    SLA_df['year_month'] = SLA_df['acceptDateTime'].dt.to_period('M')

    # Count cases per month for each user
    monthly_counts = SLA_df.groupby(['year_month', 'SLAscore']).size().reset_index(name='count')

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')
    
    # Iterate through the monthly counts and update SLA_info
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        score= row['SLAscore']
        count = row['count']
        
        # Convert year_month to datetime for column indexing
        column_date = year_month.to_timestamp()
        
        # Update the count for the specific user and month
        mask = (score_month_df['Date'] == column_date) & (score_month_df['SLAscore'] == score)
        score_month_df.loc[mask, 'case_count'] = count
    SLA_df.to_excel('excel_files/SLA_table.xlsx', index=False)
    score_month_df.to_excel('excel_files/SLA_info.xlsx', index=False)


In [7]:
def create_new_risk_status_progress_df(df):
    '''Creates a df of employee creation 'score' based on initail DB input'''

    new_df = df[(df['type']=='New') & df['customerInternalID'].notna()].copy()
    new_df['createDateTime'] = pd.to_datetime(new_df['createDateTime'])
    new_df['year_month'] = new_df['createDateTime'].dt.to_period('M')

    #Create list of unique names
    statuses = new_df['status'].dropna().unique()
    risklevels = new_df['riskLevel'].dropna().unique()
    in_progress = ['In_Progress', 'Clarification_Required', 'Outreach_Required']

    # Create range of months
    months = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')

    # Create a list of all combinations of months and persons
    data = [(month, status) for month in months for status in statuses]
    data_risk = [(month, risklevel) for month in months for risklevel in risklevels]
    data_progress = [(progress, risklevel) for progress in in_progress for risklevel in risklevels]
    
    # Create the DataFrame
    status_month_df = pd.DataFrame(data, columns=['Date', 'status'])
    risk_month_df = pd.DataFrame(data_risk, columns = ['Date', 'riskLevel'])
    risk_progress_df = pd.DataFrame(data_progress, columns = ['Progress_Status', 'riskLevel'])

    # Add the additional columns with zeros
    status_month_df['case_count'] = 0
    risk_month_df['case_count'] = 0
    risk_progress_df['case_count'] = 0
    
    # Sort the DataFrame by Month and then by Employee
    status_month_df = status_month_df.sort_values(['Date', 'status']).reset_index(drop=True)
    risk_month_df = risk_month_df.sort_values(['Date', 'riskLevel']).reset_index(drop=True)
    
    status_month_df['Date'] = pd.to_datetime(status_month_df['Date'])
    # Create year-month column
    
    # Count cases per month for each user
    monthly_counts = new_df.groupby(['year_month', 'status']).size().reset_index(name='count').copy()

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')
    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        status = row['status']
        count = row['count']
        # Convert year_month to datetime for column indexing
        column_date = year_month.to_timestamp()
        # Update the count for the specific user and month~
        mask = (status_month_df['Date'] == column_date) & (status_month_df['status'] == status) 
        status_month_df.loc[mask, 'case_count'] = count
    status_month_df['Date'] = pd.to_datetime(status_month_df['Date'])
    # Create year-month column
    
    # Count cases per month for each user
    monthly_counts = new_df.groupby(['year_month', 'riskLevel']).size().reset_index(name='count').copy()

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')
    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        risklevel = row['riskLevel']
        count = row['count']
        # Convert year_month to datetime for column indexing
        column_date = year_month.to_timestamp()
        # Update the count for the specific user and month~
        mask = (risk_month_df['Date'] == column_date) & (risk_month_df['riskLevel'] == risklevel) 
        risk_month_df.loc[mask, 'case_count'] = count

    #Create excel files
    risk_month_df.to_excel('excel_files/case_new_risk.xlsx', index=False)
    status_month_df.to_excel('excel_files/case_new_status.xlsx', index=False)
    new_df.to_excel('excel_files/case_new_table.xlsx', index=False)


In [8]:
def create_progress_no_ops_df(df):
    # Define the relevant statuses
    new_relevant_status = ['Clarification Required', 'Outreach Required', 'In Progress']
    risk_levels = ['Low', 'Standard', 'Heightened']
    
    # Initialize a list to hold DataFrames
    
    progress_data = []

    # Count cases for each status and add to the list
    for status in new_relevant_status:
        for risk_level in risk_levels:
            case_count = df[(df['status'] == status) & (df['riskLevel'] == risk_level)].shape[0]
            progress_data.append(pd.DataFrame({
                'status': [status],
                'riskLevel': [risk_level],
                'case_count': [case_count]
            }))

    #Concatenate all DataFrames in the list into a single DataFrame
    progress_df = pd.concat(progress_data, ignore_index=True)

    # Save the progress DataFrame to a new Excel file'
    progress_df.to_excel('excel_files/case_progress_risk_no_ops.xlsx', index=False)


In [9]:
def categorize_status(status):
    status_mapping = {
        "Pending QC": [
            "pending qc - or", "pending qc - completion", "qc - in progress", 
            "Pending QC - Clarification", "QC Pending Signoff"
        ],
        "Clarification Required": [
            "pending recon", "clarification required", "Clarification Required",
            "QC Review For Clarification"
        ],
        "Outreach": [
            "outreach", "Outreach Required", "QC Review for Outreach",
            "Peer Review Outreach (Opt)", "Outreach Return"
        ],
        "Under Review": [
            "under review", "or return - under review", "rework", "In Progress",
            "Peer Review Completion (Op)"
        ]
    }
    status_lower = status.lower()
    for category, status_list in status_mapping.items():
        if any(s.lower() in status_lower for s in status_list):
            return category
    return None  # For any status not categorized

#Maps statuses to categories, to be clarified later

def create_progress_ops_df(df):
    df = df.dropna(subset=['status']).copy()
    risk_levels = ['Low', 'Standard', 'Heightened']
    categories = ["Under Review", "Outreach", "Clarification Required", "Pending QC"]
    
    progress_data = []

    for category in categories:
        for risk_level in risk_levels:
            case_count = df[(df['riskLevel'] == risk_level) & (df['status'].apply(categorize_status) == category)].shape[0]
            progress_data.append(pd.DataFrame({
                'status': [category],
                'riskLevel': [risk_level],
                'case_count': [case_count]
            }))

    progress_df = pd.concat(progress_data, ignore_index=True)

    # Save the progress DataFrame to a new Excel file
    progress_df.to_excel('excel_files/case_progress_risk_ops.xlsx', index=False)
    

In [10]:
def create_jurisdiction_df(df):
    # Read the input Excel file
    
    # Drop rows with NA in 'customerInternalID'
    no_na_df = df.dropna(subset=['customerInternalID']).copy()

    # Get unique jurisdictions and their unique customerInternalID counts
    jurisdiction_counts = df.groupby('jurisdiction')['customerInternalID'].nunique().reset_index()

    jurisdiction_counts.columns = ['jurisdiction', 'unique_customer_count']
    # Add last_updated column

    # Count unique customerInternalID for each jurisdiction
    monthly_counts = no_na_df.groupby('jurisdiction')['customerInternalID'].nunique().reset_index(name='unique_customer_count')

    # Iterate through the monthly counts and update jurisdiction_info
    for _, row in monthly_counts.iterrows():
        jurisdiction = row['jurisdiction']
        count = row['unique_customer_count']
        
        # Update the count for the specific jurisdiction and date
        mask = (jurisdiction_counts['jurisdiction'] == jurisdiction)
        if mask.any():
            jurisdiction_counts.loc[mask, 'unique_customer_count'] = count
        else:
            # If the jurisdiction doesn't exist for this date, add a new row
            new_row = pd.DataFrame({
                'jurisdiction': [jurisdiction],
                'unique_customer_count': [count]
            })
            jurisdiction_counts = pd.concat([jurisdiction_counts, new_row], ignore_index=True)

    # Sort the DataFrame by jurisdiction and last_updated
    jurisdiction_counts = jurisdiction_counts.sort_values(['jurisdiction'])
    
    # Save the updated DataFrame
    jurisdiction_counts.to_excel('excel_files/jurisdiction_info.xlsx', index=False)


In [16]:
def create_completion_df(df, acc_df):
    
    last_updated = pd.to_datetime('today')  

    # Read the excel file of initial df
    
    # Create list of unique names

    # Create range of months
    # Create a list of all combinations of months and persons
    # Create the DataFrame
    relevant_status = ['Case Accepted', 'Case with Waiver Accepted', 'Case with Waiver Acceptance Required', 'Case Acceptance Required']
    
    df_com_users = df[(df['status'].isin(relevant_status)) & (df['customerInternalID'].notna())].copy()
    com_id = df_com_users['id'].tolist()
    
    for id in com_id:
        status_type = df[df['id']==id]['status'].iloc[0]

        if ((status_type == 'Case with Waiver Accepted') | (status_type == 'Case Accepted')):
            if df_com_users.loc[df_com_users['id']==id, 'type'].iloc[0] != 'New':
                if status_type == 'Case with Waiver Accepted':
                    status_from, status_to = 'Case with Waiver Acceptance Required', 'Case with Waiver Accepted'
                else:
                    status_from, status_to = 'Case Acceptance Required', 'Case Accepted'
                info = extract_status_change_info(id, status_from=status_from, status_to=status_to)
                df_com_users.loc[df_com_users['id']==id, 'accreqDateTime'] = info[4]
                df_com_users.loc[df_com_users['id']==id, 'accreqriskLevel'] = info[3]
            else:
                df_com_users.loc[df_com_users['id']==id, 'accreqDateTime'] = acc_df[acc_df['id']==id]['accreqDateTime'].iloc[0]
                df_com_users.loc[df_com_users['id']==id, 'accreqriskLevel'] = acc_df[acc_df['id']==id]['accreqriskLevel'].iloc[0]
        else:
            status_to = status_type
            info = extract_status_to_info(id, status_to=status_to)
            df_com_users.loc[df_com_users['id']==id, 'accreqDateTime'] = info[0]
            df_com_users.loc[df_com_users['id']==id, 'accreqriskLevel'] = info[2]

    
    risk_levels = df['riskLevel'].dropna().unique()
    months = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
    data = [(month, risk_level) for month in months for risk_level in risk_levels]
    risk_month_df = pd.DataFrame(data, columns=['Date', 'riskLevel'])
    risk_month_df['case_completions'] = 0
    risk_month_df = risk_month_df.sort_values(['Date', 'riskLevel']).reset_index(drop=True)
    # Create year-month column

    df_com_users['year_month'] = df_com_users['accreqDateTime'].dt.to_period('M')

    # Count cases per month for each user
    monthly_counts = df_com_users.groupby(['year_month', 'riskLevel']).size().reset_index(name='count')

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')

    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        risk_level = row['riskLevel']
        count = row['count']
        
        # Convert year_month to datetime for column indexing
        month_date = year_month.to_timestamp()
        
        # Update the count for the specific user and month
        mask = (risk_month_df['Date'] == month_date) & (risk_month_df['riskLevel'] == risk_level)
        risk_month_df.loc[mask, 'case_completions'] = count

    risk_month_df.to_excel('excel_files/case_com_info.xlsx', index=False)

    

In [12]:
def create_outreach_df(df):

    relevant_status = ['Outreach Required', 'Clarification Required']
    columns = relevant_status.copy()
    columns.append(['Cumulative OR', 'Cumulative Clarification'])
    # Create range of months
    months = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
    # Create a list of all combinations of months and persons
    data = [(month, status) for month in months for status in relevant_status]
    # Create the DataFrame
    status_month_df = pd.DataFrame(data, columns=['Date', 'outreach_type'])
    status_month_df['case_count'] = 0
    status_month_df['cumulative_count'] = 0

    df_outreach_users = df[(df['status'].isin(relevant_status)) & (df['customerInternalID'].notna())].copy()
    
    outreach_id = df_outreach_users['id'].tolist()

    for id in outreach_id:
        outreach_type = df[df['id']==id]['status'].iloc[0]
        info = extract_status_to_info(id, status_to=outreach_type)
        df_outreach_users.loc[df_outreach_users['id']==id, 'outreachDateTime'] = info[0]
        df_outreach_users.loc[df_outreach_users['id']==id, 'outreachedBy'] = info[1]
        print(outreach_type)
    
    
    df_outreach_users['outreachDateTime'] = pd.to_datetime(df_outreach_users['outreachDateTime'])
    
    def categorize_age(days):
        if days <= 29:
            return '0-29 days'
        elif days <= 59:
            return '30-59 days'
        elif days <= 89:
            return '60-89 days'
        elif days <= 119:
            return '90-119 days'
        elif days <= 149:
            return '120-149 days'
        elif days <= 179:
            return '150-179 days'
        else:
            return '180+ days'
    current_date = datetime.now()
    df_outreach_users['outreach_age'] = (current_date - df_outreach_users['outreachDateTime']).dt.days
    df_outreach_users['age_category'] = df_outreach_users['outreach_age'].apply(categorize_age)

    # Create the outreach_age_df
    age_ranges = ['0-29 days', '30-59 days', '60-89 days', '90-119 days', '120-149 days', '150-179 days', '180+ days']
    outreach_age_df = pd.DataFrame(age_ranges, columns=['Age Range'])

    # Count cases for each age range and status
    or_counts = df_outreach_users[df_outreach_users['status'] == 'Outreach Required'].groupby('age_category').size()
    cr_counts = df_outreach_users[df_outreach_users['status'] == 'Clarification Required'].groupby('age_category').size()

    # Add counts to outreach_age_df
    outreach_age_df['Count_OR'] = outreach_age_df['Age Range'].map(or_counts).fillna(0).astype(int)
    outreach_age_df['Count_Clarification'] = outreach_age_df['Age Range'].map(cr_counts).fillna(0).astype(int)

    # Save outreach_age_df to Excel
    outreach_age_df.to_excel('excel_files/case_outreach_age.xlsx', index=False)
    df_outreach_users['year_month'] = df_outreach_users['outreachDateTime'].dt.to_period('M')

    # Count cases per month for each user
    monthly_counts = df_outreach_users.groupby(['year_month', 'status']).size().reset_index(name='count')

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')
    
    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        outreach_type = row['status']
        count = row['count']
        
        # Convert year_month to datetime for column indexing
        month_date = year_month.to_timestamp()
        
        # Update the count for the specific user and month
        mask = (status_month_df['Date'] == month_date) & (status_month_df['outreach_type'] == outreach_type)
        status_month_df.loc[mask, 'case_count'] = count

    status_month_df['cumulative_count'] = status_month_df.groupby('outreach_type')['case_count'].cumsum()

    # Sort the DataFrame
    status_month_df = status_month_df.sort_values(['Date'])

    status_month_df.to_excel('excel_files/case_outreach_info.xlsx', index=False)


In [13]:
def create_refresh_df(df):
    new_df = df[(df['type']=='New') & df['customerInternalID'].notna()].copy()
    new_df['nextReviewDate'] = pd.to_datetime(new_df['nextReviewDate'], format='%d-%m-%Y')
    
    progress_stat = ['Outreach Required', 'Clarification Required', 'Case Acceptance Required', 'Case with Waiver Acceptance Required', 'Accepted', 'Accepted with Waiver', 'Cancelled']

    ref_df = df[(df['type']=='Refresh') & df['customerInternalID'].notna()].copy()
    ref_df['nextReviewDate'] = pd.to_datetime(ref_df['nextReviewDate'], format='%d-%m-%Y')
    ref_df['createDateTime'] = pd.to_datetime(ref_df['createDateTime'], format='%d-%m-%Y %H:%M:%S')

    for index, row in new_df.iterrows():
        CII = row['customerInternalID']
        matching_refresh = ref_df[(ref_df['customerInternalID'] == CII) & (~ref_df['status'].isin(progress_stat))]
        
        if not matching_refresh.empty:
            # Get the most recent Refresh case
            most_recent_refresh = matching_refresh.loc[matching_refresh['createDateTime'].idxmax()]
            # Store the ID of the most recent Refresh case
            new_df.at[index, 'actual_nextReviewDate'] = most_recent_refresh['nextReviewDate']
        else:
            new_df.at[index, 'actual_nextReviewDate'] = new_df.at[index, 'nextReviewDate']
    new_df['actual_nextReviewDate'] = pd.to_datetime(new_df['actual_nextReviewDate'], format='%d-%m-%Y')
    new_df = new_df[new_df['actual_nextReviewDate'].dt.year == 2024]
    new_df['year_month'] = new_df['actual_nextReviewDate'].dt.to_period('M')

    risklevels = new_df['riskLevel'].dropna().unique()
    
    # Create range of months
    months = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')

    # Create a list of all combinations of months and persons
    data = [(month, risk) for month in months for risk in risklevels]
    
    # Create the DataFrame
    risk_month_df = pd.DataFrame(data, columns = ['Date', 'riskLevel'])
    
    # Add the additional columns with zeros
    
    risk_month_df['case_count'] = 0
    
    # Sort the DataFrame by Month and then by Employee
    risk_month_df = risk_month_df.sort_values(['Date', 'case_count']).reset_index(drop=True)

    monthly_counts = new_df.groupby(['year_month', 'riskLevel']).size().reset_index(name='count')

    # Sort by year-month
    monthly_counts = monthly_counts.sort_values('year_month')
    
    # Iterate through the monthly counts and update count_df
    for _, row in monthly_counts.iterrows():
        year_month = row['year_month']
        risklevel = row['riskLevel']
        count = row['count']
        
        # Convert year_month to datetime for column indexing
        month_date = year_month.to_timestamp()
        
        # Update the count for the specific user and month
        mask = (risk_month_df['Date'] == month_date) & (risk_month_df['riskLevel'] == risklevel)
        risk_month_df.loc[mask, 'case_count'] = count

    risk_month_df.to_excel('excel_files/case_refresh_info.xlsx', index=False)


In [20]:
def update_file():
    """Inpute an updated excel link and update scores"""
    one_year_ago = datetime.now() - timedelta(days=200)
    karbon_df, cutoff_df = extract_karbon_excel(cutoff=one_year_ago)
    df_acc_users, df_screen_users = create_employee_metrics_df(cutoff_df)
    cutoff_df.to_excel('Test_cutoff.xlsx')
    create_SLA_df(df_acc_users)
    create_progress_no_ops_df(karbon_df)
    create_jurisdiction_df(karbon_df)
    create_completion_df(cutoff_df, df_acc_users)
    create_refresh_df(karbon_df)
    create_outreach_df(karbon_df)
    create_new_risk_status_progress_df(karbon_df)
    
update_file()

<Response [200]>
Retrieved 13 types
Fetching cases for type: New
Retrieved 400 cases for New (requested 400)
Increasing result count to 800 for New
Retrieved 800 cases for New (requested 800)
Increasing result count to 1200 for New
Retrieved 1200 cases for New (requested 1200)
Increasing result count to 1600 for New
Retrieved 1550 cases for New (requested 1600)
Total cases for New: 1550
Fetching cases for type: Refresh
Retrieved 400 cases for Refresh (requested 400)
Increasing result count to 800 for Refresh
Retrieved 655 cases for Refresh (requested 800)
Total cases for Refresh: 655
Fetching cases for type: Material Trigger Event
Retrieved 214 cases for Material Trigger Event (requested 400)
Total cases for Material Trigger Event: 214
Fetching cases for type: Non-Material Trigger Event
Retrieved 228 cases for Non-Material Trigger Event (requested 400)
Total cases for Non-Material Trigger Event: 228
Fetching cases for type: Rework
Retrieved 202 cases for Rework (requested 400)
Total ca

C:\Users\JohnElliott\AppData\Local\Temp\ipykernel_15732\2170061829.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A (Created as Case Accepted)' has dtype incompatible with timedelta64[us], please explicitly cast to a compatible dtype first.
  df_acc_users.loc[df_acc_users['id']==id, 'acceptTimeDelta'] = info[1]


Case ID: 2f5f1799-7740-49d1-bc0b-c1e898e4278c
Time from Case with Waiver Acceptance Required to Case with Waiver Accepted: N/A (Created as Case with Waiver Accepted)
Date status moved to Case with Waiver Accepted: 2024-09-04 11:42:31
Updated By: None
Risk Level: Low

Case with Waiver Accepted
Case ID: 415176f5-d709-4f75-bd29-5da78baf1597
Time from Case with Waiver Acceptance Required to Case with Waiver Accepted: N/A (Created as Case with Waiver Accepted)
Date status moved to Case with Waiver Accepted: 2024-09-04 11:42:34
Updated By: None
Risk Level: Low

Case with Waiver Accepted
Case ID: d7076972-68b5-4bf3-b87c-57b425153e39
Time from Case Acceptance Required to Case Accepted: N/A (Created as Case Accepted)
Date status moved to Case Accepted: 2024-09-04 09:19:56
Updated By: None
Risk Level: Low

Case Accepted
Case ID: 23d9599e-45fe-4a5c-b5f6-f2b582e0b7a0
Time from Case Acceptance Required to Case Accepted: N/A (Created as Case Accepted)
Date status moved to Case Accepted: 2024-09-04 0